In [1]:
!pip install -U \
  transformers==4.37.2 \
  peft==0.9.0 \
  accelerate==0.27.2 \
  datasets==2.15.0 \
  evaluate==0.4.0 \
  seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 108.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.5 MB/s eta 0:00:00
  

In [2]:
from huggingface_hub import login


In [3]:
from datasets import load_dataset

# ds_es = load_dataset("wikiann", "es")
# ds_en = load_dataset("wikiann", "en")
# ds_ja = load_dataset("wikiann", "ja")
ds_es = load_dataset("wikiann", name="es", split="train[:2000]")
ds_en = load_dataset("wikiann", name="en", split="train[:2000]")
ds_ja = load_dataset("wikiann", name="ja", split="train[:2000]")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

...

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [4]:
def get_small_subset(ds, size=2000):
    return ds.shuffle(seed=42).select(range(size))

# ds_train = get_small_subset(ds_es) + get_small_subset(ds_en) + get_small_subset(ds_ja)
# ds_val = get_small_subset(ds_es, 500) + get_small_subset(ds_en, 500) + get_small_subset(ds_ja, 500)

from datasets import concatenate_datasets
ds_train = concatenate_datasets([ds_es, ds_en, ds_ja])
ds_val = concatenate_datasets([
    ds_es.shuffle(seed=123).select(range(500)),
    ds_en.shuffle(seed=123).select(range(500)),
    ds_ja.shuffle(seed=123).select(range(500))
])

In [5]:
from transformers import AutoTokenizer

model_checkpoint = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
label_list = ds_train.features["ner_tags"].feature.names

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
from transformers import AutoModelForTokenClassification
from peft import get_peft_model, LoraConfig, TaskType

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

peft_config = LoraConfig(
    task_type=TaskType.TOKEN_CLS,
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 300,295 || all params: 277,758,734 || trainable%: 0.10811361200976673


In [7]:
from transformers import DataCollatorForTokenClassification
from transformers import TrainingArguments, Trainer
import evaluate
import numpy as np

data_collator = DataCollatorForTokenClassification(tokenizer)

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if True else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_train = ds_train.map(tokenize_and_align_labels, batched=True)
tokenized_val = ds_val.map(tokenize_and_align_labels, batched=True)


Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [10]:
args = TrainingArguments(
    "xlmr-ner-lora",
    evaluation_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=10,
    report_to="none",
    fp16=True,  #No quiero usar esto pero no me queda mas remedio, reduce el 50% de uso de ram
    load_best_model_at_end=True

)

import evaluate
metric = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = [[label_list[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return metric.compute(predictions=true_predictions, references=true_labels)


In [11]:
from transformers import EarlyStoppingCallback

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)] #Para que no sobreajuste y si despues de 2 etapas no mejora que pare

)

trainer.train()

/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py:450: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss,Loc,Org,Per,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,0.700900,0.553046,"{'precision': 0.47371364653243847, 'recall': 0.5035671819262783, 'f1': 0.4881844380403459, 'number': 1682}","{'precision': 0.3226480836236934, 'recall': 0.42244525547445255, 'f1': 0.3658632951402607, 'number': 1096}","{'precision': 0.4797486033519553, 'recall': 0.7097107438016529, 'f1': 0.5725, 'number': 968}",0.429001,0.533102,0.475420,0.801323
2,0.483500,0.495730,"{'precision': 0.5831894070236039, 'recall': 0.6022592152199763, 'f1': 0.5925709271716876, 'number': 1682}","{'precision': 0.5059055118110236, 'recall': 0.46897810218978103, 'f1': 0.48674242424242425, 'number': 1096}","{'precision': 0.5367192862045299, 'recall': 0.8078512396694215, 'f1': 0.6449484536082474, 'number': 968}",0.548456,0.616391,0.580442,0.829448
3,0.616600,0.398864,"{'precision': 0.6352413019079686, 'recall': 0.6730083234244947, 'f1': 0.653579676674365, 'number': 1682}","{'precision': 0.5438441098317095, 'recall': 0.5602189781021898, 'f1': 0.5519101123595507, 'number': 1096}","{'precision': 0.7032085561497327, 'recall': 0.8150826446280992, 'f1': 0.7550239234449762, 'number': 968}",0.628564,0.676722,0.651755,0.858711
4,0.440600,0.362920,"{'precision': 0.6720670391061453, 'recall': 0.7152199762187872, 'f1': 0.6929723502304147, 'number': 1682}","{'precision': 0.5493527508090615, 'recall': 0.6195255474452555, 'f1': 0.5823327615780446, 'number': 1096}","{'precision': 0.6817800167926112, 'recall': 0.8388429752066116, 'f1': 0.7522000926354794, 'number': 968}",0.638843,0.719167,0.676629,0.864347
5,0.444900,0.344831,"{'precision': 0.6837606837606838, 'recall': 0.713436385255648, 'f1': 0.6982833866744254, 'number': 1682}","{'precision': 0.5916808149405772, 'recall': 0.6359489051094891, 'f1': 0.6130167106420404, 'number': 1096}","{'precision': 0.6913265306122449, 'recall': 0.8398760330578512, 'f1': 0.7583955223880596, 'number': 968}",0.659528,0.723438,0.690006,0.872999
6,0.384200,0.330152,"{'precision': 0.718384074941452, 'recall': 0.7294887039239001, 'f1': 0.7238938053097345, 'number': 1682}","{'precision': 0.5640287769784172, 'recall': 0.7153284671532847, 'f1': 0.6307320997586483, 'number': 1096}","{'precision': 0.7430426716141002, 'recall': 0.8274793388429752, 'f1': 0.782991202346041, 'number': 968}",0.673372,0.750667,0.709922,0.879825
7,0.418300,0.294852,"{'precision': 0.7420515896820636, 'recall': 0.7354340071343638, 'f1': 0.738727978501045, 'number': 1682}","{'precision': 0.6078431372549019, 'recall': 0.7071167883211679, 'f1': 0.6537326022775201, 'number': 1096}","{'precision': 0.7726829268292683, 'recall': 0.8181818181818182, 'f1': 0.794781736076267, 'number': 968}",0.706831,0.748532,0.727084,0.897870
8,0.528000,0.287745,"{'precision': 0.7397576457010964, 'recall': 0.7621878715814506, 'f1': 0.750805270863836, 'number': 1682}","{'precision': 0.6379746835443038, 'recall': 0.6897810218978102, 'f1': 0.6628671635247698, 'number': 1096}","{'precision': 0.7792836398838335, 'recall': 0.8316115702479339, 'f1': 0.8045977011494254, 'number': 968}",0.719565,0.758943,0.738729,0.901468
9,0.300300,0.284165,"{'precision': 0.7374301675977654, 'recall': 0.7847800237812128, 'f1': 0.7603686635944701, 'number': 1682}","{'precision': 0.6451612903225806, 'recall': 0.6751824817518248, 'f1': 0.6598305840392331, 'number': 1096}","{'precision': 0.7753554502369668, 'recall': 0.8450413223140496, 'f1': 0.8086999505684627, 'number': 968}",0.720942,0.768286,0.743861,0.900939
10,0.196100,0.277669,"{'precision': 0.7481565513329552, 'recall': 0.7841854934601664, 'f1': 0.7657474600870828, 'number': 1682}","{'precision': 0.6392887383573242, 'recall': 0.6888686131386861, 'f1': 0.663153271848924, 'number': 1096}","{'precision': 0.7805343511450382, 'recall': 0.8450413223140496, 'f1': 0.8115079365079365, 'number': 968}",0.724449,0.772023,0.747480,0.904352


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in ver

TrainOutput(global_step=7500, training_loss=0.48609634351730346, metrics={'train_runtime': 525.198, 'train_samples_per_second': 114.243, 'train_steps_per_second': 14.28, 'total_flos': 2969844772141536.0, 'train_loss': 0.48609634351730346, 'epoch': 10.0})

In [12]:
from transformers import pipeline

ner_pipeline = pipeline(
    "ner",
    model=trainer.model,        # Usa el modelo directamente desde el Trainer
    tokenizer=tokenizer,
    aggregation_strategy="simple"
)

text = "Pedro Sánchez se reunió en Bruselas con Ursula von der Leyen el 4 de mayo de 2023."

entities = ner_pipeline(text)

for ent in entities:
    print(f"Entidad: {ent['word']} | Tipo: {ent['entity_group']} | Score: {ent['score']:.2f}")


The model 'PeftModelForTokenClassification' is not supported for ner. Supported models are ['AlbertForTokenClassification', 'BertForTokenClassification', 'BigBirdForTokenClassification', 'BioGptForTokenClassification', 'BloomForTokenClassification', 'BrosForTokenClassification', 'CamembertForTokenClassification', 'CanineForTokenClassification', 'ConvBertForTokenClassification', 'Data2VecTextForTokenClassification', 'DebertaForTokenClassification', 'DebertaV2ForTokenClassification', 'DistilBertForTokenClassification', 'ElectraForTokenClassification', 'ErnieForTokenClassification', 'ErnieMForTokenClassification', 'EsmForTokenClassification', 'FalconForTokenClassification', 'FlaubertForTokenClassification', 'FNetForTokenClassification', 'FunnelForTokenClassification', 'GPT2ForTokenClassification', 'GPT2ForTokenClassification', 'GPTBigCodeForTokenClassification', 'GPTNeoForTokenClassification', 'GPTNeoXForTokenClassification', 'IBertForTokenClassification', 'LayoutLMForTokenClassification'

Entidad: Pedro | Tipo: LABEL_1 | Score: 0.97
Entidad: Sánchez | Tipo: LABEL_2 | Score: 0.98
Entidad: se reunió en | Tipo: LABEL_0 | Score: 1.00
Entidad: Bruselas | Tipo: LABEL_5 | Score: 0.68
Entidad: con | Tipo: LABEL_0 | Score: 1.00
Entidad: Ursula | Tipo: LABEL_1 | Score: 0.99
Entidad: von der Leyen | Tipo: LABEL_2 | Score: 0.99
Entidad: el 4 de mayo de 2023. | Tipo: LABEL_0 | Score: 0.99


In [13]:
text = "岸田文雄首相はワシントンでジョー・バイデン米大統領と会談した。" #"Prime Minister Kishida Fumio met with U.S. President Joe Biden in Washington."
entities = ner_pipeline(text)

for ent in entities:
    print(f"Entidad: {ent['word']} | Tipo: {ent['entity_group']} | Score: {ent['score']:.2f}")

Entidad:  | Tipo: LABEL_0 | Score: 0.99
Entidad: 岸田 | Tipo: LABEL_1 | Score: 0.86
Entidad: 文雄首相 | Tipo: LABEL_2 | Score: 0.93
Entidad: は | Tipo: LABEL_0 | Score: 1.00
Entidad: ワシントン | Tipo: LABEL_5 | Score: 0.45
Entidad: で | Tipo: LABEL_0 | Score: 1.00
Entidad: ジョー | Tipo: LABEL_1 | Score: 0.96
Entidad: ・バイデン米大統領 | Tipo: LABEL_2 | Score: 0.93
Entidad: と会談した。 | Tipo: LABEL_0 | Score: 1.00


In [14]:
# Entidad: | Tipo: LABEL_0 | Score: 0.99
# Entidad: Kishida | Tipo: LABEL_1 | Score: 0.89
# Entidad: Prime Minister Fumio | Tipo: LABEL_2 | Score: 0.95
# Entidad: is | Tipo: LABEL_0 | Score: 1.00
# Entidad: Washington | Tipo: LABEL_5 | Score: 0.58
# Entidad: in | Tipo: LABEL_0 | Score: 0.96
# Entidad: Joe | Tipo: LABEL_1 | Score: 0.89
# Entidad: US President Biden | Tipo: LABEL_2 | Score: 0.88
# Entidad: met with. | Tipo: LABEL_0 | Score: 0.99

In [15]:
for idx, label in enumerate(label_list):
    print(f"LABEL_{idx} → {label}")

LABEL_0 → O
LABEL_1 → B-PER
LABEL_2 → I-PER
LABEL_3 → B-ORG
LABEL_4 → I-ORG
LABEL_5 → B-LOC
LABEL_6 → I-LOC


In [ ]:
# Okey, so know that I have (mostly) a functional model, let's text how good it is with 3 tests:
# - Text so long
# - Text with languages mixed
# - Text with a language no trained

In [ ]:
text1 = (
    "Pedro Sánchez se reunió con Ursula von der Leyen en Bruselas para discutir los planes de recuperación. "
    "Durante la conferencia, también estuvieron presentes representantes de Alemania, Francia, Italia y Portugal. "
    "El acuerdo alcanzado tendrá implicaciones importantes para el futuro de la Unión Europea, especialmente en áreas como "
    "la sostenibilidad, digitalización y cohesión social. Mientras tanto, en Japón, el primer ministro Fumio Kishida comentó sobre la situación global..."
    * 5
)
text2 = "Pedro Sánchez met Ursula von der Leyen in Brussels. 同時に、岸田文雄首相は東京で発言しました。 President Biden also commented on the situation."
text3 = "Emmanuel Macron a rencontré Angela Merkel à Strasbourg pour discuter de l'avenir de l'Europe."


In [ ]:
import time

text = text1

start = time.time()
entities = ner_pipeline(text)
end = time.time()

print(f"⏱ Tiempo total de inferencia: {end - start:.2f} segundos")
for ent in entities:
    print(f"Entidad: {ent['word']} | Tipo: {ent['entity_group']} | Score: {ent['score']:.2f}")

In [ ]:
import time

text = text2

start = time.time()
entities = ner_pipeline(text)
end = time.time()

print(f"⏱ Tiempo total de inferencia: {end - start:.2f} segundos")
for ent in entities:
    print(f"Entidad: {ent['word']} | Tipo: {ent['entity_group']} | Score: {ent['score']:.2f}")

In [ ]:
import time

text = text3

start = time.time()
entities = ner_pipeline(text)
end = time.time()

print(f"⏱ Tiempo total de inferencia: {end - start:.2f} segundos")
for ent in entities:
    print(f"Entidad: {ent['word']} | Tipo: {ent['entity_group']} | Score: {ent['score']:.2f}")

## Voy a probar a conseguir una web para poder hacer esto dinámico

In [16]:
import gradio as gr
from transformers import pipeline

ner_pipeline = pipeline(
    "ner",
    model=model,           # Ya definido
    tokenizer=tokenizer,   # Ya definido
    aggregation_strategy="simple"
)

def extract_entities(text):
    entities = ner_pipeline(text)
    if not entities:
        return "No se encontraron entidades."

    output = "Entidades detectadas:\n"
    for ent in entities:
        output += f"🔹 {ent['word']} → {ent['entity_group']} (confianza: {ent['score']:.2f})\n"
    return output

#Gradio
demo = gr.Interface(
    fn=extract_entities,
    inputs=gr.Textbox(lines=5, label="Introduce un texto"),
    outputs=gr.Textbox(label="Entidades reconocidas"),
    title="Reconocimiento de Entidades (NER)",
    description="Modelo multilingüe fine-tuneado sobre XLM-Roberta con LoRA.",
    theme="default"
)

demo.launch(share=True)

The model 'PeftModelForTokenClassification' is not supported for ner. Supported models are ['AlbertForTokenClassification', 'BertForTokenClassification', 'BigBirdForTokenClassification', 'BioGptForTokenClassification', 'BloomForTokenClassification', 'BrosForTokenClassification', 'CamembertForTokenClassification', 'CanineForTokenClassification', 'ConvBertForTokenClassification', 'Data2VecTextForTokenClassification', 'DebertaForTokenClassification', 'DebertaV2ForTokenClassification', 'DistilBertForTokenClassification', 'ElectraForTokenClassification', 'ErnieForTokenClassification', 'ErnieMForTokenClassification', 'EsmForTokenClassification', 'FalconForTokenClassification', 'FlaubertForTokenClassification', 'FNetForTokenClassification', 'FunnelForTokenClassification', 'GPT2ForTokenClassification', 'GPT2ForTokenClassification', 'GPTBigCodeForTokenClassification', 'GPTNeoForTokenClassification', 'GPTNeoXForTokenClassification', 'IBertForTokenClassification', 'LayoutLMForTokenClassification'

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://41052d989292356cf6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
